<a href="https://colab.research.google.com/github/YuvrajChauhan1303/C-Programs/blob/main/DAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import find_peaks, welch
import neurokit2 as nk



In [ ]:
# Step 1: Read the CSV and flatten the signal
df = pd.read_csv("a01.csv", header=None, dtype=str)
df = df.replace(r'[^\d\.\-eE]', '', regex=True)  # Remove non-numeric artifacts
df = df.apply(pd.to_numeric, errors='coerce')   # Convert to numeric
df = df.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)  # Forward/back fill NaNs
ecg_data = df.values.flatten()

# Sampling frequency
fs = 100

In [ ]:
# Step 2: Clean ECG and extract features
signals, info = nk.ecg_process(ecg_data, sampling_rate=fs)
rpeaks = info['ECG_R_Peaks']
rr_intervals = np.diff(rpeaks) / fs
hr = 60 / rr_intervals

In [ ]:


# Time-domain features
features = {
    "Mean": np.mean(ecg_data),
    "Std": np.std(ecg_data),
    "Var": np.var(ecg_data),
    "Median": np.median(ecg_data),
    "Min": np.min(ecg_data),
    "Max": np.max(ecg_data),
    "Range": np.max(ecg_data) - np.min(ecg_data),
    "Q1": np.percentile(ecg_data, 25),
    "Q3": np.percentile(ecg_data, 75),
    "IQR": np.percentile(ecg_data, 75) - np.percentile(ecg_data, 25),
    "Skewness": skew(ecg_data),
    "Kurtosis": kurtosis(ecg_data),
    "RMS": np.sqrt(np.mean(np.square(ecg_data))),
    "Zero_Crossings": ((ecg_data[:-1] * ecg_data[1:]) < 0).sum(),
    "Signal_Energy": np.sum(ecg_data ** 2),
    "Signal_Entropy": entropy(np.histogram(ecg_data, bins=50, density=True)[0]),
    "Mean_Absolute_Diff": np.mean(np.abs(np.diff(ecg_data))),
    "STD_Absolute_Diff": np.std(np.abs(np.diff(ecg_data))),
    "Symmetry_Index": np.mean(ecg_data[:len(ecg_data)//2]) - np.mean(ecg_data[len(ecg_data)//2:]),
}

# Frequency-domain features using Welch's method
frequencies, power = welch(ecg_data, fs, nperseg=1024)
features.update({
    "Total_Power": np.trapz(power, frequencies),
    "VLF_Power": np.trapz(power[(frequencies >= 0.003) & (frequencies < 0.04)], frequencies[(frequencies >= 0.003) & (frequencies < 0.04)]),
    "LF_Power": np.trapz(power[(frequencies >= 0.04) & (frequencies < 0.15)], frequencies[(frequencies >= 0.04) & (frequencies < 0.15)]),
    "HF_Power": np.trapz(power[(frequencies >= 0.15) & (frequencies < 0.4)], frequencies[(frequencies >= 0.15) & (frequencies < 0.4)]),
})

# LF/HF Ratio
lf = features["LF_Power"]
hf = features["HF_Power"]
features["LF_HF_Ratio"] = lf / hf if hf != 0 else np.nan

# HRV features
features.update({
    "Num_Beats": len(rpeaks),
    "Mean_RR": np.mean(rr_intervals),
    "STD_RR": np.std(rr_intervals),
    "Min_RR": np.min(rr_intervals),
    "Max_RR": np.max(rr_intervals),
    "Mean_HR": np.mean(hr),
    "SDNN": np.std(rr_intervals),
    "RMSSD": np.sqrt(np.mean(np.square(np.diff(rr_intervals)))),
    "NN50": np.sum(np.abs(np.diff(rr_intervals)) > 0.05),
    "pNN50": np.sum(np.abs(np.diff(rr_intervals)) > 0.05) / len(rr_intervals) * 100,
})

# Save features to CSV
features_df = pd.DataFrame([features])
features_df.to_csv("extracted_ecg_features.csv", index=False)
print("✅ Features extracted and saved to 'extracted_ecg_features.csv'")
